In [ ]:
import os
import re
import sys
import h5py
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
from pathlib import Path

if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from pfcommon import OU, combine_output_spectra
from filter_OU_inputs import run_welch

FIGURES_DIR = Path('figures')

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.major', **{'width': lw, 'size': 3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size': 1.5})

In [ ]:
use_dBs = False
dB = 20
dP = 0.01
# the name of the stochastic load
stoch_load = 'Load_03'
load_type = 'general_load'
load_type = 'static_load_const_Z'
short_sim = True
assert dP >= 0.01
suffix = '_{:.02f}'.format(dP)
network_name = 'IEEE39_stoch'

The folder where data are stored:

In [ ]:
base_data_dir = Path('../data')
data_dir = base_data_dir / 'IEEE39_stoch' / 'default' / load_type / stoch_load
assert os.path.isdir(data_dir)
tran_file = '{}_tran{}{}.npz'.format(network_name, suffix, '_short' if short_sim else '')
assert os.path.isfile(data_dir / tran_file)

Load the data from the transient simulation performed by PowerFactory:

In [ ]:
tran_data = np.load(data_dir / tran_file, allow_pickle=True)
data = tran_data['data'].item()

Make sure that the generators' speeds are constant around 1 p.u. (i.e., the network is stable):

In [ ]:
t_tran = tran_data['time']
gen_speed = data['gen']['s:xspeed']
fig,ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(t_tran, gen_speed, lw=0.75)
ax.set_xlabel('Time [s]')
ax.set_ylabel(r'$\omega_{gen}$ [p.u.]')
sns.despine()
fig.tight_layout()

#### Loads checks

Here I perform the following checks:
1. the generated active power of the load should match the one recorded from PF.
2. the PSD of both time series should match the theoretical one.

The PSD of an OU process is given by
$$
\mathrm{PSD}(f) = \frac{(\frac{c}{\alpha})^2}{1 + (\frac{2\pi f}{\alpha})^2},
$$
where $f$ is the frequency, $\alpha=1/\tau$, with $\tau$ the autocorrelation time constant of the process and $c=\sigma \sqrt{2/\tau}$, with $\sigma$ the (steady-state) standard deviation of the process.

In [ ]:
stoch_loads = tran_data['stoch_load_names'].tolist()
assert len(stoch_loads) == 1 and stoch_loads[0] == stoch_load
OU_seed = tran_data['OU_seeds'][0]
P0 = tran_data['stoch_load_P'][0]
Q0 = tran_data['stoch_load_Q'][0]
print(f'Load mean P: {P0:g} MW.')
print(f'Load mean Q: {Q0:g} MVAR.')
print(f'Seed: {OU_seed}.')

In [ ]:
config = tran_data['config'].item()
dt_tran = config['dt']
μ = P0                        # mean
σ = config['sigma']['P'] * μ  # standard deviation
τ = config['tau']['P']        # time constant
c = σ * np.sqrt(2 / τ)
α = 1 / τ
rs = RandomState(MT19937(SeedSequence(OU_seed)))
N_samples = int(np.ceil(config['tstop'] / dt_tran)) + 1
t_OU = np.arange(N_samples) * dt_tran
P_OU = OU(dt_tran, μ, σ, τ, N_samples, rs)
P_OU_from_file = tran_data['OU_P'].squeeze()
assert np.allclose(P_OU, P_OU_from_file), 'Data from file does not match the generated OU time series'

In [ ]:
device_names = tran_data['device_names'].item()
if data['bus']['m:U'].ndim == 1:
    U, phiu = data['bus']['m:U'], np.deg2rd(data['bus']['m:phiu'])
else:
    idx = device_names['bus'].index(stoch_load.replace('Load','Bus'))
    U, phiu = data['bus']['m:U'][:,idx], np.deg2rad(data['bus']['m:phiu'][:,idx])
if data['load']['m:I:bus1'].ndim == 1:
    I, phii = data['load']['m:I:bus1'], np.deg2rad(data['load']['m:phii:bus1'])
    P_from_PF = data['load']['m:Psum:bus1']
    Q_from_PF = data['load']['m:Qsum:bus1']
else:
    idx = device_names['load'].index(stoch_load)
    I, phii = data['load']['m:I:bus1'][:,idx], np.deg2rad(data['load']['m:phii:bus1'][:,idx])
    P_from_PF = data['load']['m:Psum:bus1'][:,idx]
    Q_from_PF = data['load']['m:Qsum:bus1'][:,idx]
u = U * np.exp(1j * phiu)
i = I * np.exp(1j * phii)
S = 3 * u * i.conjugate()
P_from_PF_2 = S.real
Q_from_PF_2 = S.imag
assert np.allclose(P_from_PF, P_from_PF_2) and np.allclose(Q_from_PF, Q_from_PF_2)
Z = u / i
Z_abs = np.abs(Z)

In [ ]:
Fmin, Fmax = -6, 2
one_sided = True
freq = np.logspace(Fmin, Fmax, 50*(Fmax-Fmin+1))
PSD_theor = (c / α) ** 2 / (1 + (2 * np.pi * freq / α) ** 2)
freq_OU,PSD_OU,abs_OU = run_welch(P_OU - μ, dt_tran, window=50/dt_tran, onesided=one_sided)
freq_from_PF,PSD_from_PF,abs_from_PF = run_welch(P_from_PF - P_from_PF.mean(), dt_tran, window=50/dt_tran, onesided=one_sided)

In [ ]:
green = [.2,.8,.2]
magenta = [.8,.2,.8]
fig,ax = plt.subplots(3, 1, figsize=(5,5))
tstop = 5
jdx = t_tran <= tstop
ax[0].plot([0, tstop], μ + np.zeros(2), 'k', lw=2, label='Pmean')
ax[0].plot(t_tran[jdx], P_from_PF[jdx], color=green, lw=0.75, label='From PF')
ax[0].plot(t_tran[jdx], P_from_PF_2[jdx], '--', color='tab:red', lw=0.75, label='From PF - 2')
ax[1].plot(t_tran[jdx], Z_abs[jdx], 'k', lw=0.75)
jdx = t_OU <= tstop
ax[0].plot(t_OU[jdx], P_OU[jdx], color=magenta, lw=1, label='OU computed from seed')
ax[0].legend(loc='best', frameon=True, fontsize=fontsize-2)
ax[0].set_xlabel('Time [s]')
ax[1].set_xlabel('Time [s]')
ax[0].set_ylabel('P [MW]')
ax[1].set_ylabel('|Z| [Ω]')
ax[2].plot(freq_from_PF, PSD_from_PF, color=green, lw=0.5, label='From PF')
ax[2].plot(freq_OU, PSD_OU, color=magenta, lw=0.5, label='From file')
ax[2].plot(freq, PSD_theor, 'k', lw=2, label='Theory')
ax[2].set_xscale('log')
ax[2].set_xlabel('Frequency [Hz]')
ax[2].set_ylabel(r'|Y(j$\omega$)|')
ax[2].set_xlim([1e-2, 10**Fmax])
sns.despine()
fig.tight_layout()
outfile = 'loads_and_spectra_{}_{}_{}{}.pdf'.format(network_name, load_type, stoch_load, suffix)
print(f"Saving to file '{outfile}'")
plt.savefig(FIGURES_DIR / outfile)

Compute the spectra of the data obtained from the transient simulation:

In [ ]:
var_group_tran = 'bus'
if var_group_tran == 'gen':
    device_name_tran = 'G_01'
    var_type_tran = 's:xspeed'
elif var_group_tran == 'bus':
    device_name_tran = stoch_load.replace('Load', 'Bus')
    var_type_tran = 'm:ur'
    # var_type_tran = 'm:ui'
    # var_type_tran = 'm:u'
idx = device_names[var_group_tran].index(device_name_tran)
x_tran = data[var_group_tran][var_type_tran][:,idx]
if var_type_tran == 'm:u':
    ur = data[var_group_tran]['m:ur'][:,idx]
    ui = data[var_group_tran]['m:ui'][:,idx]
    x_tran_check = np.sqrt(ur**2 + ui**2)
    assert np.allclose(x_tran, x_tran_check)
jdx = t_tran > 100
x_tran = x_tran[jdx]
t_tran = t_tran[jdx]
Δx_tran = x_tran - x_tran.mean()
freq_tran,P_tran,abs_tran = run_welch(Δx_tran, dt_tran, window=10/dt_tran, onesided=one_sided)

if use_dBs:
    abs_tran = dB * np.log10(abs_tran)
    ylbl = r'|Y(j$\omega$)| [dB{}]'.format(dB)
else:
    ylbl = r'|Y(j$\omega$)|'

Load additional information about the transfer functions:

In [ ]:
steps_per_decade = 100
TF_file = '{}_TF_-6.0_2.0_{}_{:.2f}.npz'.format(network_name, steps_per_decade, dP)
assert os.path.isfile(data_dir / TF_file)
TF_data = np.load(data_dir / TF_file, allow_pickle=True)
PF = TF_data['PF'].item()

Load the data from the small-signal simulation, i.e., the variables obtained by filtering the input(s) with the appropriate transfer functions:

In [ ]:
small_signal_file = '{}_ss{}.h5'.format(network_name, suffix)
fid = h5py.File(data_dir / small_signal_file)
t_ss = np.array(fid['time'])
def read_list(fid, key):
    names = fid['parameters'][key].tolist()[0]
    return list(map(lambda n: n.decode('utf-8'), names))
load_names = read_list(fid, 'load_names')
var_names_ss = read_list(fid, 'var_names')

if var_type_tran == 's:xspeed':
    var_type_ss = 'speed'
elif var_type_tran in ('m:ur', 'm:ui'):
    var_type_ss = var_type_tran[2:]
elif var_type_tran == 'm:u':
    var_type_ss = 'U'
else:
    raise Exception("Unknown variable type '{var_type_tran}'")
if var_group_tran == 'bus':
    var_group_ss = 'Term'
elif var_group_tran == 'gen':
    var_group_ss = 'Sym'
else:
    raise Exception("Unknown variable group '{var_group_tran}'")
var_name_ss = 'Grid_{}_Elm{}_{}'.format(device_name_tran, var_group_ss, var_type_ss)
print(var_name_ss)
var_names_ss.append('Grid-{}.Elm{}.{}'.format(device_name_tran, var_group_ss, var_type_ss))

if var_type_ss != 'U':
    x_ss = np.array(fid[var_name_ss]).squeeze()
else:
    ur, ui = PF['buses'][device_name_tran]['ur'], PF['buses'][device_name_tran]['ui']
    coeff_ur, coeff_ui = np.array([ur, ui]) / np.sqrt(ur**2 + ui**2)
    x_ss = coeff_ur * np.array(fid[var_name_ss[:-1] + 'ur']).squeeze() + \
           coeff_ui * np.array(fid[var_name_ss[:-1] + 'ui']).squeeze()
fid.close()

Build the theoretical PSDs of the output variables:

In [ ]:
all_var_names = TF_data['var_names'].tolist()
all_load_names = TF_data['load_names'].tolist()
F0 = 50.
var_types = []
F = TF_data['F']
N_vars = len(var_names_ss)
for i in range(N_vars):
    _,typ = os.path.splitext(var_names_ss[i])
    if typ == '.ur':
        var_types.append('m:ur')
    elif typ == '.ui':
        var_types.append('m:ui')
    elif typ == '.U':
        var_types.append('U')
    elif typ == '.speed':
        var_types.append('s:xspeed')
    elif typ == '.fe':
        var_types.append('m:fe')
    else:
        raise Exception(f"Unknown variable type '{typ[1:]}'")
OUT_multi = combine_output_spectra(TF_data['OUT'], load_names, var_names_ss, all_load_names,
                                   all_var_names, var_types, F, PF,
                                   TF_data['bus_equiv_terms'].item(), ref_freq=F0)
out = np.abs(OUT_multi)
if use_dBs:
    out = dB*np.log10(out)

Compute the spectra of the data obtained from the small signal simulation:

In [ ]:
dt_ss = t_ss[1] - t_ss[0]
Δx_ss = x_ss - x_ss.mean()
freq_ss,P_ss,abs_ss = run_welch(Δx_ss, dt_ss, window=100/dt_ss, onesided=one_sided)
if use_dBs:
    abs_ss = dB * np.log10(abs_ss)

In [ ]:
from scipy.integrate import trapezoid
idx = [n.replace('.','_').replace('-','_') for n in var_names_ss].index(var_name_ss)
print('\n===== Small signal data =====')
print('Integral of the theoretical PSD: {:g}'.format(trapezoid(out[idx]**2 * (1 + one_sided), F)))
print('Integral of the numerical PSD: {:g}'.format(trapezoid(P_ss * (1 + one_sided), freq_ss)))
print('Variance of the signal: {:g}'.format(np.var(x_ss)))
print('\n===== PF transient data =====')
print('Integral of the PSD: {:g}'.format(trapezoid(P_tran * (1 + one_sided), freq_tran)))
print('Variance of the signal: {:g}'.format(np.var(x_tran)))

In [ ]:
typ = var_name_ss.split('_')[-1]
fig,ax = plt.subplots(2, 1, figsize=(5,5))

remove_mean = True
if remove_mean:
    ax[0].plot(t_tran, Δx_tran, 'k', lw=0.75, label='Tran')
    ax[0].plot(t_ss, Δx_ss, 'tab:green', lw=0.75, label='S.S.', alpha=0.75)
    ax[0].set_ylabel(f'Δ{typ} [p.u.]')
else:
    twin_ax = ax[0].twinx()
    ax[0].plot(t_tran, x_tran, 'k', lw=0.75, label='Tran')
    twin_ax.plot(t_ss, x_ss, 'tab:green', lw=0.75, label='S.S.', alpha=0.75)
    ax[0].set_ylabel(r'${}_t$ [p.u.]'.format(typ))
    twin_ax.set_ylabel(r'${}_s$ [p.u.]'.format(typ))
ax[0].set_xlabel('Time [s]')

ax[1].plot(freq_tran, abs_tran, 'k', lw=0.75, label=r'Tran ($\sigma^2$ = {:.1e})'.\
               format(trapezoid(P_tran * (1 + one_sided), freq_tran)))
ax[1].plot(freq_ss, abs_ss, 'tab:green', lw=0.75, label='S.S. ($\sigma^2$ = {:.1e})'.\
               format(trapezoid(P_ss * (1 + one_sided), freq_ss)), alpha=0.75)
ax[1].plot(F, out[idx], 'tab:red', lw=2, label='Theory')
loc = 'lower left' if use_dBs else 'best'
ax[1].legend(loc=loc, frameon=False, fontsize=fontsize-1)
ax[1].set_xscale('log')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel(ylbl)
ax[1].set_xlim([1e-3, F[-1]])
if remove_mean:
    sns.despine()
fig.tight_layout()
outfile = 'spectra_comparison_tran_ss_{}_{}_{}{}_{}_{}.pdf'.format(network_name, load_type, stoch_load, suffix, device_name_tran.split('__')[0], typ)
print(f"Saving to file '{outfile}'")
plt.savefig(FIGURES_DIR / outfile)